<a href="https://colab.research.google.com/github/chiangtinhung-rgb/Matcha/blob/main/HW5_%E5%8D%88%E9%A4%90_%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8%EF%BC%88%E5%B8%B6%E6%A2%9D%E4%BB%B6%E9%81%8E%E6%BF%BE%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **午餐/聚會決策器（帶條件過濾）（作業五）**

目標：從 Sheet 讀餐廳清單（價位/距離/是否營業）→ 篩選 → 隨機推薦 3 家 → 寫建議清單。

AI 點子：把 3 家優缺點壓成 100 字說明，幫助快速決策。

1. I/O to GoogleSheet(自行評估需不需要google sheet輔助)

2. 修改老師原本的code範例，改成自己的版本

3. 地圖(map)放入

4. 將map 匯入gradio


In [1]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=6fb54982d0a48506f8266bcf94fab25834078002b65f74743a7076724636001b
  Stored in directory: /root/.cache/pip/wheels/4c/6a/a7/bbc6f5c200032025ee655deb5e163ce8594fa05e67d973aad6
Successfully built googlemaps


In [ ]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil

In [ ]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [ ]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1myM3mFwjKdn0aJdENwhue18m-gwCDFLoPKpMqw6t1L0/edit?usp=sharing"
WORKSHEET_NAME = "模擬資料"
TIMEZONE = "Asia/Taipei"

In [ ]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

In [2]:
# 安裝： pip install googlemaps
import googlemaps
import os

API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")  # 建議從環境變數讀取
gmaps = googlemaps.Client(key=API_KEY)

# 1. 設定搜尋位置（範例：台北市中心）
lat, lng = 25.0330, 121.5654
radius = 1000  # 公尺

# 2. Nearby Search 呼叫
results = gmaps.places_nearby(
    location=(lat, lng),
    radius=radius,
    type='restaurant',  # 專找餐廳
    open_now=False      # 可選：只找目前開放
)

for place in results.get('results', []):
    name = place.get('name')
    address = place.get('vicinity')
    place_id = place.get('place_id')
    rating = place.get('rating')
    print(f"Name: {name}, Address: {address}, Rating: {rating}, Place ID: {place_id}")

    # 若想拿更多細節，可呼叫 Place Details
    details = gmaps.place(
        place_id=place_id,
        fields=['name','formatted_address','formatted_phone_number','website','rating','user_ratings_total']
    )
    deta = details.get('result', {})
    print("   Phone:", deta.get('formatted_phone_number'))
    print("   Website:", deta.get('website'))
    print("   Total Ratings:", deta.get('user_ratings_total'))

ValueError: Must provide API key or enterprise credentials when creating client.

In [ ]:
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import json

# --- 2. 核心功能函數 ---

def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""

    # 轉換篩選條件為列表
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    # Gradio CheckboxGroup 回傳列表，但單一勾選時需處理
    available_status = ['是'] if is_available else ['是', '否']

    # 應用篩選條件
    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    # 隨機抽取 3 家，不足 3 家則全數取出
    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    # 準備推薦清單文字
    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    # 準備 AI 摘要的輸入字串
    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


async def generate_summary(ai_input):
    """呼叫 LLM 產生決策建議摘要 (使用 gemini-2.5-pro)"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    # Fallback if model configuration failed
    await asyncio.sleep(1) # 模擬網路延遲
    mock_summary = f"""
    **AI 助理 (模擬):** 今天的選擇是：{'、'.join([r.split('(')[0].strip() for r in ai_input.split('、')])}。
    如果您追求**效率和銅板價**，{ai_input.split('、')[0].split('，')[0]} (價位{ai_input.split('、')[0].split('，')[1].split('價位')[1]}) 肯定是首選，快速又飽足。
    若想來點**儀式感和高品質**，{'、'.join(ai_input.split('、')[1:])} 提供了更好的用餐體驗，但距離和花費會稍高一點。
    決定權在你！
    (請確保您已安裝 'google-genai' 並在 Colab Secrets 中設置 'gemini' API key 以啟用真實 AI 推薦)
    """
    return mock_summary.strip()


    system_prompt = (
        "您是一位幽默且極具效率的午餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、距離、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要，不超過 150 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的餐廳清單：{ai_input}。請綜合考量其價格、距離和特色，為我提供一份快速決策建議。"

    try:
        # 使用 asyncio.to_thread 讓同步的 SDK 呼叫不會阻塞 Gradio 的事件迴圈
        response = await asyncio.to_thread(
            model.generate_content,
            contents=user_query,
            config=genai.types.GenerateContentConfig(
                system_instruction=system_prompt,
                temperature=0.7,
            )
        )
        return response.text.strip()

    except Exception as e:
        error_msg = f"呼叫 Gemini API 失敗，請檢查 API 金鑰或配額：{e}。"
        return error_msg


# 介面更新函數
async def wrapper_recommend(price_filter, distance_filter, is_available):
    """Gradio 主要流程：篩選 -> 推薦 -> 呼叫 AI 摘要"""

    # 1. 篩選與推薦
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_restaurants, price_filter, distance_filter, is_available
    )

    # 2. 呼叫 AI 摘要
    ai_summary = await generate_summary(ai_input)

    # 3. 準備給 Gradio 輸出
    # 將推薦結果轉換為 Dataframe (僅顯示名稱、價位、距離)
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    # 處理空結果
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）"

    return display_df, recommend_list, ai_summary


# --- 3. Gradio 介面配置 ---

# 設定 Gradio 介面元件
with gr.Blocks(title="迷你黑客松午餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 迷你黑客松午餐/聚會決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家，最後由 AI 助您快速決策！
        **🚨 注意：請確保您已在 Colab 中讀取 Google Sheet 數據，否則將使用模擬數據。**
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 條件篩選")
            price_select = gr.CheckboxGroup(
                ['$', '$$', '$$$'],
                label="價位過濾 (可複選)",
                value=['$', '$$'],
                info="($: 銅板/150內, $$: 150-300, $$$: 300+)"
            )
            distance_select = gr.CheckboxGroup(
                ['短', '中', '長'],
                label="距離過濾 (可複選)",
                value=['短', '中'],
                info="(短: 步行5分, 中: 步行10分, 長: 需騎車)"
            )
            available_toggle = gr.Checkbox(
                label="只看今日營業",
                value=True
            )
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=2):
            gr.Markdown("## 推薦清單 (隨機 3 家)")
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                datatype=["str", "str", "str"],
                label="隨機推薦結果"
            )
            output_list = gr.Textbox(label="推薦清單細節")

    gr.Markdown("## 🤖 AI 決策建議 (約 100 字)")
    output_summary = gr.Textbox(
        label="AI 決策顧問報告",
        interactive=False,
        info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
    )

    # 定義點擊按鈕後的行為 (使用異步包裝函數)
    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[
            price_select,
            distance_select,
            available_toggle
        ],
        outputs=[output_dataframe, output_list, output_summary]
    )

    gr.Markdown(
        """
        ---
        **[開發者備註]** AI 摘要已設定為使用 `gemini-2.5-pro` 呼叫。請確保您已執行以下步驟：
        1.  在 Colab 中安裝 `google-genai` 等必要套件。
        2.  在 Colab Secrets 中設定名為 `gemini` 的 API 金鑰。
        """
    )


# 運行 Gradio 應用
if __name__ == "__main__":
    # 在 Colab 或一般環境中運行
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True)